In [13]:
import requests
import json #Importa las librerías necesarias:
from IPython.display import JSON
from keys import *

In [17]:

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/RKPK/2025-09-01T08:00/2025-09-01T20:00"

#se definen los parámetros de la consulta
querystring = {
    "offsetMinutes": "-120",
    "durationMinutes": "720",
    "withLeg": "true",
    "direction": "Both",
    "withCancelled": "true",
    "withCodeshared": "true",
    "withCargo": "true",
    "withPrivate": "true",
    "withLocation": "false"
}
#se definen los encabezados HTTP
headers = {
    "x-rapidapi-host": "aerodatabox.p.rapidapi.com",
    "x-rapidapi-key": "cbae0c8b00msha53f5d91d17f5f2p1c7e20jsn6f6909a55841"  
}
#hace petición GET a la API
response = requests.get(url, headers=headers, params=querystring)

#se imprime el código de estado HTTP de la petición
print("Status code:", response.status_code)
print(response.json())


Status code: 200
{'departures': [{'departure': {'scheduledTime': {'utc': '2025-08-31 23:00Z', 'local': '2025-09-01 08:00+09:00'}, 'terminal': 'I', 'quality': ['Basic']}, 'arrival': {'airport': {'icao': 'VVTS', 'iata': 'SGN', 'name': 'Ho Chi Minh City', 'timeZone': 'Asia/Ho_Chi_Minh'}, 'scheduledTime': {'utc': '2025-09-01 03:55Z', 'local': '2025-09-01 10:55+07:00'}, 'terminal': '1', 'quality': ['Basic']}, 'number': 'VJ 869', 'callSign': 'VJC869', 'status': 'Unknown', 'codeshareStatus': 'IsOperator', 'isCargo': False, 'aircraft': {'reg': 'VN-A649', 'modeS': '888157', 'model': 'Airbus A321'}, 'airline': {'name': 'VietJetAir', 'iata': 'VJ', 'icao': 'VJC'}}, {'departure': {'scheduledTime': {'utc': '2025-08-31 23:00Z', 'local': '2025-09-01 08:00+09:00'}, 'terminal': 'D', 'quality': ['Basic']}, 'arrival': {'airport': {'icao': 'RKPC', 'iata': 'CJU', 'name': 'Jeju City', 'timeZone': 'Asia/Seoul'}, 'scheduledTime': {'utc': '2025-09-01 00:05Z', 'local': '2025-09-01 09:05+09:00'}, 'quality': ['Bas

In [18]:
if response.status_code == 200: #validación que la respuesta de la API fue exitosa
    new_data = response.json() #convierte la respuesta en diccionario de Python y lo guarda

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = [] #si el archivo no existe o está vacío/dañado, entonces existing_data se inicializa como una lista vacía

    existing_data.append(new_data)#agrega la nueva respuesta

    with open("data.json", "w") as json_file: #se abre data.json en modo escritura y guarda todo el contenido de existing_data
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.") #imprime un mensaje confirmando que los datos fueron agregados
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code) #si la API no responde con éxito, se muestra un mensaje de error 

Data appended to data.json file.


In [19]:
flight_arrivals = response.json() #se obtiene la respuesta de la API en formato JSON

# Recorremos cada vuelo en la lista de 'arrivals'
for flight in flight_arrivals.get('arrivals', []):
    scheduled_time = flight.get('arrival', {}).get('scheduledTime')
    flight_number = flight.get('number')
    terminal = flight.get('arrival', {}).get('terminal')
    from_airport = flight.get('departure', {}).get('airport', {}).get('name')
    aircraft = flight.get('aircraft', {}).get('model')
    airline = flight.get('airline', {}).get('name')

    # Imprimimos toda la información del vuelo
    print("Hora programada:", scheduled_time)
    print("Número de vuelo:", flight_number)
    print("Terminal:", terminal)
    print("Aeropuerto de origen:", from_airport)
    print("Avión:", aircraft)
    print("Aerolínea:", airline)
    print("-" * 40)


Hora programada: {'utc': '2025-08-31 23:05Z', 'local': '2025-09-01 08:05+09:00'}
Número de vuelo: KE 1803
Terminal: D
Aeropuerto de origen: Seoul-si
Avión: Airbus A220-300
Aerolínea: Korean Air
----------------------------------------
Hora programada: {'utc': '2025-08-31 23:10Z', 'local': '2025-09-01 08:10+09:00'}
Número de vuelo: BX 602
Terminal: I
Aeropuerto de origen: Denpasar-Bali Island
Avión: Airbus A321 NEO
Aerolínea: Air Busan
----------------------------------------
Hora programada: {'utc': '2025-08-31 23:10Z', 'local': '2025-09-01 08:10+09:00'}
Número de vuelo: 7C 2662
Terminal: I
Aeropuerto de origen: Singapore
Avión: Boeing 737-800
Aerolínea: Jeju Air
----------------------------------------
Hora programada: {'utc': '2025-08-31 23:30Z', 'local': '2025-09-01 08:30+09:00'}
Número de vuelo: 7C 2552
Terminal: I
Aeropuerto de origen: Bangkok
Avión: Boeing 737-800
Aerolínea: Jeju Air
----------------------------------------
Hora programada: {'utc': '2025-08-31 23:40Z', 'local': '

In [20]:
import requests
import pandas as pd #importa las librerías necesarias:

#convierte la respuesta de la API (response) en un diccionario Python
flight_arrivals = response.json()

#define una función que extrae la información importante de un solo vuelo
def flight_extraction(flight):
    record = {
        'scheduled_arrival_time': flight.get('arrival', {}).get('scheduledTime'),
        'flight_number': flight.get('number'),
        'from_airport': flight.get('departure', {}).get('airport', {}).get('name'),
        'airline': flight.get('airline', {}).get('name'),
        'aircraft': flight.get('aircraft', {}).get('model')
    }
    return pd.DataFrame([record])  # cada vuelo como DataFrame de 1 fila

# Convierte la lista de arrivals en DataFrame
flight_arrivals_df = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals.get('arrivals', [])],
    ignore_index=True
)

# Mostrar las primeras filas
print(flight_arrivals_df.head())


                              scheduled_arrival_time flight_number  \
0  {'utc': '2025-08-31 23:05Z', 'local': '2025-09...       KE 1803   
1  {'utc': '2025-08-31 23:10Z', 'local': '2025-09...        BX 602   
2  {'utc': '2025-08-31 23:10Z', 'local': '2025-09...       7C 2662   
3  {'utc': '2025-08-31 23:30Z', 'local': '2025-09...       7C 2552   
4  {'utc': '2025-08-31 23:40Z', 'local': '2025-09...        VJ 990   

           from_airport     airline         aircraft  
0              Seoul-si  Korean Air  Airbus A220-300  
1  Denpasar-Bali Island   Air Busan  Airbus A321 NEO  
2             Singapore    Jeju Air   Boeing 737-800  
3               Bangkok    Jeju Air   Boeing 737-800  
4             Nha Trang  VietJetAir      Airbus A321  
